In [1]:
import numpy as np

sem_dir = "../sab_work/"
out_dir = "../data/sem_data/"
words = ["accident", "brilliant", "float", "excess", "promise"]
remap_words = ["accident-n", "brilliant-a", "float-v", "excess-n", "promise-v"]
remap_dict = {}
for i in range(len(words)):
    remap_dict[words[i]] = remap_words[i]
sets = ['train', 'test']

In [8]:
###get data in and map to a dictionary of dictionaries. Format is dictionary[feature][set] where feature is the word
###and set is like accident_train which returns the list of rows that have that feature in that set

word = words[0]
set_type = sets[0]

main_dict = {}
total_rows = {}
for word in words:
    for set_type in sets:
        with open(sem_dir + "sem_" +  word + "_" + set_type + ".txt") as f:
            num_rows = 0
            for i, line in enumerate(f.readlines()):
                for w1 in line.split(" "):
                    second_level = word + "_" + set_type
                    feature = w1.strip()
                    
                    main_dict[word] = main_dict.get(word, {})
                    main_dict[word][feature] = main_dict[word].get(feature, {})
                    main_dict[word][feature][set_type] = main_dict[word][feature].get(set_type, [])
                    main_dict[word][feature][set_type].append(i)
                    
                num_rows += 1
            total_rows[word + "_" + set_type] = num_rows
total_cols = {}
for w in words:
    total_cols[w] = len(main_dict[w].keys())

In [22]:
###explode the dictionary format to numpy arrays of same number of columns

n_features = len(main_dict.keys())
np_dict = {}
for s in total_rows.items():
    w =  s[0].split("_")[0]
    np_dict[s[0]] = np.zeros((s[1], total_cols[w]))

for word in main_dict.keys():
    for i, feature in enumerate(main_dict[word]):
        for set_type in main_dict[word][feature]:
            np_array = np_dict[word + "_" + set_type]
            for row in main_dict[word][feature][set_type]:
                np_array[row, i] = 1
            np_dict[word + "_" + set_type] = np_array

In [23]:
###write to disk

for data_set in np_dict.items():
    word_type = remap_dict[data_set[0].split("_")[0]]
    set_type = data_set[0].split("_")[1]
    np.save(out_dir + word_type + "_" + set_type + "_sem.npy", data_set[1])

In [28]:
import pandas as pd
x=pd.DataFrame(np.load("../data/sem_data/promise-v_test_sem.npy"))

In [29]:
x.sum(axis=1).value_counts()

13    21
14    15
9     15
12    14
11    14
17    13
15    13
8     12
10    11
6      9
19     9
7      8
16     8
18     8
20     7
5      6
23     6
22     4
21     4
3      4
4      3
2      3
25     3
27     3
29     2
32     2
41     1
24     1
28     1
31     1
34     1
35     1
54     1
dtype: int64